In [31]:

import spacy
import pandas as pd
import json
import os


In [158]:
df=pd.read_csv("dictionaries\Distribution_descriptors.csv",header=None)
df1=pd.read_csv("dictionaries\Life_stages.csv",header=None)
df2=pd.read_csv("dictionaries\Body_size.csv",header=None)
#df3=pd.read_csv("dictionaries\biological_descriptors.csv",header=None,encoding='cp1252')

df_distr_descr=df.iloc[:,0]
df_life_stages=df1.iloc[:,0]
df_body_size=df2.iloc[:,0]
#df_biol_descr=df3.iloc[:,0]
#to lige stages ousiastika einai mesa sta biological descriptors



In [159]:
from spacy.pipeline import EntityRuler
from spacy.lang.en import English

#nlp=English()
#arxikopoihsh tou nlp kai entity ruler
nlp=spacy.blank("en")
ruler=nlp.add_pipe("entity_ruler")

def dict_func(df,linkdf,label):
    patterns=[]
    j=0
    for i in df:
        dict={"label":label}
        dict["pattern"]=i #[{"LOWER" : str.lower(i)}]
        print(str.lower(i))
        dict["id"]=linkdf[[1]][1][j]
        patterns.append(dict)
        j=j+1
    ruler.add_patterns(patterns)

# def dict_func(df,linkdf,label):
#     patterns=[]
#     j=0
#     for i in df:
#         dict={"label":label}
#         dict["pattern"]=[{"LOWER" : {"REGEX": str.lower(i) }}] #+ "?" to ? sto telos tou string dinei dunatothta na pianei oles tis pithanes lekseis,alla kanei lathh,px gia to apples tha piasei kai to stapples h se emas apo to Mud tha piasei kai to museum ara de mas kanei,pros to paron vriskei oles tis dunates allages vash tou caps
#         dict["id"]=linkdf[[1]][1][j]
#         patterns.append(dict)
#         j=j+1
#     ruler.add_patterns(patterns)

 
tokenizer = nlp.tokenizer
tokens = tokenizer(df_distr_descr[14])
print(tokens)
len(tokens)


Artificial (man-made)


6

In [160]:
dict_func(df_distr_descr,df,"Distribution Descriptor")
dict_func(df_life_stages,df1,"Life Stage")
dict_func(df_body_size,df2,"Body Size")
#dict_func(df_biol_descr,"Biological Descriptor")


abyssobenthic
abyssal
abyssopelagic
alphamesohaline
alpha-mesohaline
alphaoligohaline
alpha-oligohaline
alpjapolyhaline
alpha-polyhaline
alphahaline
alpha-haline
anchialinecaves
anchialine caves
artificial
artificial (man-made)
bathybenthic
bathybenthic (bathyal)
bathylpelagic
bathypelagic(1000-2500m)
bedrock
benthic
benthopelagic
betahaline
beta-haline
betamesohaline
beta-mesohaline
betaoligohaline
beta-oligohaline
betapolyhaline
beta-polyhaline
biogenicreef
biogenic reef
brackish
brackishsalinity
burrowdweller
burrow dweller
cave
circalittoral
circalittoraloffshore
circalittoral offshore
clay
corarsecleansand
coarse clean sand
corarsesediments
coarse sediments
cobbles
crevicesfissures
crevices and fissures
demersal
depth
depthsubstratum
depth in substratum
depthrange
depth range
elevation
enclosedcoast
encloast coast or embayment
endophytic
endozoic
environment
environmentalposition
epibenthic
epifununal
epifloral
epilithic
epipelagic
epipelagic(0-2000m)
epipelic
epiphytic
epizooic
e

In [201]:
#edw vazoume keimeno gia dictionary testing 

doc=nlp("Atherinids are small Marine estuary and freshwaters Alpha-Polyhaline Body length (SL) standard length (sl) fishes not exceeding 120 mm SL (a soon to be described species of Craterocephalus may reach 300 mm SL), occurring predominantly in the Old World, with only Alepidomus evermanni (freshwaters of Cuba) and two marine species, Atherinomorus stipes and Hypoatherina harringtonensis predominantly in the shore waters of the Caribbean) known from the New World. In the Southeast Pacific, the family is only known only from Tumaco, Colombia, according to Dyer. All freshwater atherinid species are derived from marine ancestors and until recently have only been known from the rivers of Australia, New Guinea and some islands in the vicinity of the latter.")
paper=nlp("(SL), head length and eye diameter. Counts were madeusingthetechniquerecommendedbyHubbs & Lagler (1958) so as to make comparisons with previously described atherinid species. Two speci�mens were cleared and stained usingthe method of Taylor (1967), Taylor & Vandyke (1985) and Dingerkus & Uhler (1977). These were measured and counted prior to clearing. Drawings were made with the assistance of a stereomicroscope. Osteological comparisons were made with repre�sentatives of marine Indo-Pacific atherinds and an atherionid (Table II). Specimens collected by H. Bleher�s party were deposited in the Australian Museum Sydney, those borrowed from the MNHN were returned to that institution. All of the specimens that have been examined are desig�nated as types.")

print([(ent.text, ent.label_ ,ent.start_char,ent.end_char, ent.ent_id_) for ent in doc.ents])


[('Salinity', 'Distribution Descriptor', 11, 19, 'https://www.marinespecies.org/traits/wiki/Traits:Salinity'), ('Marine', 'Distribution Descriptor', 30, 36, 'https://www.marinespecies.org/traits/wiki/Traits:Marine'), ('Alpha-Polyhaline', 'Distribution Descriptor', 61, 77, 'https://www.marinespecies.org/traits/wiki/Traits:AlphaPolyhaline'), ('Body length', 'Body Size', 78, 89, 'https://www.marinespecies.org/traits/wiki/Traits:Bodylength')]


In [162]:
import random
import spacy.cli
from spacy.tokens import DocBin
from tqdm import tqdm

In [120]:
TRAIN_DATA = [ ("interations between infaunal species which can attain high densities within cages.",{"entities":[(20,29,"Distribution_descriptor"),(76,81,"Distribution_descriptor")]}),
("infauna, they also provide a good opportunity to study competitive or adult-larval ",{"entities":[(0,7,"Distribution_descriptor"),(70,75,"Life_stage"),(76,82,"Life_stage"),(70,82,"Life_stage")]}),
(" on ",{"entities":[]}),
("influence of larger predators such as fish, crabs, and epibenthic predatory benthos on",{"entities":[(76,84,"Distribution_descriptor"),(55,66,"Distribution_descriptor")]}),
("with those in control plots. Although cages are most commonly used to assess the ",{"entities":[]}),
("or in which densities of such adults are varied, and to compare the larval densities there ",{"entities":[(30,36,"Life_stage"),(68,75,"Life_stage")]}),
("establish plots from which adults which may interact with settling larvae are removed ",{"entities":[(27,33,"Life_stage"),(67,73,"Life_stage")]}),
("The basic procedure in field experiments examining adult-larval interactions is to ",{"entities":[(51,56,"Life_stage"),(57,63,"Life_stage"),(51,63,"Life_stage")]}),
("and some islands in the vicinity of the latter.",{"entities":[]}),
("known from the rivers of Australia, New Guinea ",{"entities":[(15,22,"Distribution_descriptor")]}),
("marine ancestors and until recently have only been ",{"entities":[(0,7,"Distribution_descriptor")]}),
("All freshwater atherinid species are derived from ",{"entities":[(4,15,"Distribution_descriptor")]}),
("from Tumaco, Colombia, according to Dyer. ",{"entities":[]}),
("Southeast Pacific, the family is only known only ",{"entities":[]}),
("Caribbean) known from the New World. In the ",{"entities":[]}),
("(predominantly in the shore waters of the ",{"entities":[(22,34,"Distribution_descriptor")]}),
("Atherinomorus stipes and Hypoatherina harringtonensis ",{"entities":[]}),
("World, with only Alepidomus evermanni (freshwaters of Cuba) and two marine species, ",{"entities":[(39,51,"Distribution_descriptor"),(17,37,"Distribution_descriptor"),(68,74,"Distribution_descriptor")]}),
("be described species of Craterocephalus may reach 300 mm SL), occurring predominantly in the Old ",{"entities":[(50,59,"Body_size"),(57,59,"Body_size"),(50,56,"Body_size")]}),
("Atherinids are small marine, estuarine and freshwater fishes not exceeding 120 mm SL (a soon to ",{"entities":[(21,27,"Distribution_descriptor"),(29,39,"Distribution_descriptor"),(43,54,"Distribution_descriptor"),(82,84,"Body_size"),(75,81,"Body_size")]}),
("lagoon population but a potential contribution to the genetic flux among coastal populations. ",{"entities":[(0,7,"Distribution_descriptor"),(73,81,"Distribution_descriptor")]}),
("the estuarine larvae that drifted out of the lagoon seemed to survive, suggesting a loss of individuals from the ",{"entities":[(4,14,"Distribution_descriptor"),(14,21,"Life_stage"),(45,52,"Distribution_descriptor")]}),
("CW to LR, possibly reducing their survival. Generally, eggs were more vulnerable to drift than larvae. Many of ",{"entities":[(0,2,"Distribution_descriptor"),(6,8,"Distribution_descriptor"),(55,60,"Life_stage"),(95,101,"Life_stage")]}),
("when heavy rains led to severe lagoon discharges. Similarly, unusual strong onshore winds advected eggs from ",{"entities":[(5,16,"Distribution_descriptor"),(31,37,"Distribution_descriptor"),(76,89,"Distribution_descriptor"),(99,104,"Life_stage")]}),
("larvae to LR. Eggs and larvae of estuarine species that spawned within LR were effectively retained there, except ",{"entities":[(0,6,"Life_stage"),(13,18,"Life_stage"),(22,29,"Life_stage"),(32,42,"Distribution_descriptor"),(70,73,"Distribution_descriptor")]}),
("transport of these larvae into LR. Conversely, there was no evidence of entrance of marine estuarine-independent ",{"entities":[(19,25,"Life_stage"),(31,33,"Distribution_descriptor"),(91,100,"Distribution_descriptor"),(91,112,"Distribution_descriptor"),(84,90,"Distribution_descriptor")]}),
("and light onshore winds during the open-sandbar period were the most important process that promoted the ",{"entities":[(40,47,"Distribution_descriptor"),(10,23,"Distribution_descriptor")]}),
("abundance in CW. Sandbar closure did not impede the entrance of these species. Overwashing events (closed bar) ",{"entities":[(13,15,"Distribution_descriptor"),(17,24,"Distribution_descriptor")]}),
("runoff) and species-specific traits (e.g. life cycle strategy, spawning season) determined ichthyoplankton exchange vs. retention. Marine estuarine-related species entered LR mostly during summer-autumn, at their peak ",{"entities":[(0,6,"Distribution_descriptor"),(91,106,"Life_stage"),(131,137,"Distribution_descriptor"),(138,147,"Distribution_descriptor"),(138,155,"Distribution_descriptor"),(172,174,"Distribution_descriptor")]}),
("nursery ground. Larval stages of six marine species were found at CW. Environmental factors (e.g. wind and ",{"entities":[(16,23,"Life_stage"),(36,43,"Distribution_descriptor"),(64,66,"Distribution_descriptor"),(96,100,"Distribution_descriptor")]}),
("exchange of water and ichthyoplankton. At least three estuarine species were found to spawn and use LR as a ",{"entities":[(12,17,"Distribution_descriptor"),(22,37,"Life_stage"),(54,64,"Distribution_descriptor"),(100,102,"Distribution_descriptor")]}),
("conditions and ichthyoplankton communities between LR and CW, and also evidenced different mechanisms of ",{"entities":[(51,54,"Distribution_descriptor"),(57,59,"Distribution_descriptor")]}),
("along a year in LR and CW to capture different sandbar states (open/closed) and lagoon-sea connectivity processes (e.g. lagoon discharges, marine intrusion, overwashing). Results showed differences in environmental ",{"entities":[(16,18,"Distribution_descriptor"),(23,25,"Distribution_descriptor"),(47,55,"Distribution_descriptor"),(80,90,"Distribution_descriptor"),(119,126,"Distribution_descriptor"),(137,144,"Distribution_descriptor")]}),
("waters (CW), and the influence of that connection on ichthyoplankton assemblages. We performed field surveys ",{"entities":[(0,7,"Distribution_descriptor"),(8,10,"Distribution_descriptor")]}),
("This study evaluated the connectivity between an intermittently closed-open lagoon (LR) and adjacent coastal ",{"entities":[(84,86,"Body_size"),(76,83,"Distribution_descriptor"),(101,108,"Distribution_descriptor"),(92,108,"Distribution_descriptor"),(64,82,"Distribution_descriptor"),(84,86,"Distribution_descriptor")]}),
("spawned previously. A summary of the life-cycle is also ",{"entities":[]}),
("indicated that some females in this sample had ",{"entities":[]}),
("(8.8%). A few atretic oocytes and empty follicles ",{"entities":[]}),
("stage (91.2%) or in the rst stage of vitellogenesis ",{"entities":[(37,51,"Body_size")]}),
("and spermatozoa. Oocytes were in the previtellogenic ",{"entities":[]}),
("sample were in a resting stage. Testes lacked spermatids ",{"entities":[]}),
("histology indicated that the gonads of this November ",{"entities":[]}),
("Although specimens were large and sexually mature, the ",{"entities":[]}),
("index (IG) was 0.9% for females and 0.2% for males. ",{"entities":[]}),
("1.6% for females and 1.7% for males; gonadosomatic ",{"entities":[]}),
("relationship to W and LT. Hepatosomatic index (IH) was ",{"entities":[(16,17,"Body_size"),(22,24,"Body_size")]}),
("showed no difference due to sex, but a signicant weak ",{"entities":[]}),
("Subcutaneous lipid thickness averaged 2.6 mm and ",{"entities":[(19,29,"Body_size"),(38,44,"Body_size")]}),
("26.7 kg (range 10.460.3 kg). The length-weight relationship was W=3.44 105 L2:85T (n 28, r2 0.96). ",{"entities":[(0,7,"Body_size"),(9,26,"Body_size"),(0,27,"Body_size"),(33,46,"Body_size"),(33,39,"Body_size"),(40,46,"Body_size"),(65,71,"Body_size"),(74,84,"Body_size")]}),
("in McMurdo Sound, Antarctica. Total length (LT) averaged 127.3 cm (range 90162 cm) and weight (W) was ",{"entities":[(30,42,"Body_size"),(44,46,"Body_size"),(30,47,"Body_size"),(57,65,"Body_size"),(67,81,"Body_size"),(87,97,"Body_size"),(87,93,"Body_size"),(94,97,"Body_size")]}),
("capture on a set line near the southern limit of its range ",{"entities":[]}),
("Twenty-eight specimens of the large not othenioid Dissostichus mawsoni were dissected after ",{"entities":[]}),
("species.",{"entities":[]}),
("appears to be influenced by conflicting selection pressures that may differ between the ",{"entities":[]}),
("R. whero and competition for habitat in both species. Body size in the Ruanoho species ",{"entities":[(29,37,"Distribution_descriptor"),(54,63,"Body_size")]}),
("size differences in the Ruanoho pair are consistent with female choice for smaller males in ",{"entities":[(0,5,"Body_size")]}),
("while female R. decemdigitatus had no preference for male size. Results suggest that body ",{"entities":[(58,62,"Body_size")]}),
("in heterospecific trials. Female R. where showed a strong preference for smaller males, ",{"entities":[]}),
("a subset of that displayed by R. decemdigitatus, while no mating behaviour was observed ",{"entities":[(58,64,"Distribution_descriptor")]}),
("R. whero were displaced into less preferred habitats. Courtship behaviour in R. whero was ",{"entities":[(44,52,"Distribution_descriptor")]}),
("dominant in inter- and intraspecific contests for the preferred rock habitat, while small ",{"entities":[(64,76,"Distribution_descriptor")]}),
("trials. Both species exhibited similar habitat preferences, but large R. decemdigitatus were ",{"entities":[(39,47,"Distribution_descriptor"),(39,58,"Distribution_descriptor"),(64,70,"Body_size")]}),
("setting. Reproductive behaviour was assessed in courtship, mate choice and hybridisation ",{"entities":[(9,31,"Distribution_descriptor"),(48,57,"Distribution_descriptor"),(75,88,"Distribution_descriptor")]}),
("trials based on the field data to test habitat preference and competitive ability in a common ",{"entities":[(39,47,"Distribution_descriptor")]}),
("substratum. The effect of body size on substratum use was investigated using laboratory ",{"entities":[(0,10,"Distribution_descriptor"),(26,35,"Body_size"),(39,50,"Distribution_descriptor")]}),
("20 m. In sites where the species co-occurred, R. whero was less associated with rock ",{"entities":[(0,4,"Body_size"),(80,84,"Distribution_descriptor")]}),
("while R. whero was frequently found in sheltered to moderately exposed areas down to ",{"entities":[]}),
("distributions and differ in body size, attaining a total length of 12 cm and 9 cm, respectively. R. decemdigitatus was most commonly found in sheltered areas shallower than 5 m, ",{"entities":[(28,37,"Body_size"),(51,63,"Body_size"),(67,72,"Body_size"),(77,81,"Body_size"),(51,72,"Body_size"),(51,81,"Body_size"),(142,157,"Distribution_descriptor")]}),
("New Zealand. Ruanoho decemdigitatus and Ruanoho whero have overlapping sympatric ",{"entities":[]}),
("The effect of body size on spatial resource competition and reproductive isolation was examined in a sister species pair of subtidal triplefin fishes (F. Tripterygiidae) in ",{"entities":[(14,23,"Body_size")]}),
("fin height and eye size, which are invariably greater in P. muelleri. ",{"entities":[(4,11,"Body_size"),(15,23,"Body_size")]}),
("albilabris are similar in appearance and habits, but are separable on the basis of dorsal ",{"entities":[(41,47,"Distribution_descriptor")]}),
("by its black colouration, longer dorsal fin , longer barbels, and greater number oflowerlimb gill rakers and upper procurrent caudal rays. Paraplotosus muelleri and P. ",{"entities":[(7,24,"Distribution_descriptor")]}),
("from coastal seas of northwestern Australia. It is clearly distinguishable from congeners ",{"entities":[(5,17,"Distribution_descriptor"),(5,12,"Distribution_descriptor")]}),
("butleri, new species, is described from 40 specimens, 26-325 mm SL, collected mainly ",{"entities":[(54,66,"Body_size"),(64,66,"Body_size"),(57,66,"Body_size")]}),
("Northern Territory, and an undescribed species from northern Australia. Paraplotosus ",{"entities":[]}),
("Archipelago, P. muelleri (Klunzinger) from north-western Australia and adjacent ",{"entities":[]}),
("species are recognised: P. albilabris (Valenciennes) from the Indo-Australian ",{"entities":[]}),
("The marine catfish genus Paraplotosus Bleeker is reviewed. Three ",{"entities":[(4,11,"Distribution_descriptor")]}),
]


In [121]:
#in order to create training data,custom ner model
nlp.create_pipe("ner")
nlp.add_pipe("ner",last=True)


ValueError: [E007] 'ner' already exists in pipeline. Existing names: ['entity_ruler', 'ner']

In [181]:
nlp.pipe_names

['tok2vec', 'ner', 'entity_ruler']

In [123]:
doc_bin = DocBin()

In [124]:
from spacy.util import filter_spans

for training_example in tqdm(TRAIN_DATA): 
    text = training_example[0]
    labels = training_example[1]['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

100%|██████████| 79/79 [00:00<00:00, 2323.75it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [125]:
!python -m spacy init fill-config base_config.cfg config.cfg


✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [126]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     51.00    0.00    0.00    0.00    0.00
 16     200         28.96   1598.18   97.80   98.23   97.37    0.98
 37     400         11.75     76.75  100.00  100.00  100.00    1.00
 63     600         14.68     37.47   98.25   98.25   98.25    0.98
 94     800         29.53     73.14   98.69   98.26   99.12    0.99
133    1000         27.57     44.87  100.00  100.00  100.00    1.00
180    1200         36.87     44.17  100.00  100.00  100.00    1.00
236    1400         22.35     27.37  100.00  100.00  100.00    1.00
303    1600         12.56      9.01 

[2022-11-21 14:02:28,751] [INFO] Set up nlp object from config
[2022-11-21 14:02:28,760] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-11-21 14:02:28,763] [INFO] Created vocabulary
[2022-11-21 14:02:31,184] [INFO] Added vectors: en_core_web_lg
[2022-11-21 14:02:33,894] [INFO] Finished initializing nlp object
[2022-11-21 14:02:34,270] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [220]:
nlp_ner=spacy.load("model-best")

In [221]:

doc=nlp_ner("Atherinids adult larval are small marine estuary and freshwaters Alpha-Polyhaline Body length (SL) standard length (sl) fishes not exceeding 150 mm SL (a soon to be described species of Craterocephalus may reach 300 mm SL), occurring predominantly in the Old World, with only Alepidomus evermanni (freshwaters of Cuba) and two marine species, Atherinomorus stipes and Hypoatherina harringtonensis predominantly in the shore waters of the Caribbean) known from the New World. In the Southeast Pacific, the family is only known only from Tumaco, Colombia, according to Dyer. All freshwater atherinid species are derived from marine ancestors and until recently have only been known from the rivers of Australia, New Guinea and some islands in the vicinity of the latter.")
paper=nlp_ner("(SL), marine Salinity head length and eye diameter. Counts were madeusingthetechniquerecommendedbyHubbs & Lagler (1958) so as to make comparisons with previously described atherinid species. Two speci�mens were cleared and stained usingthe method of Taylor (1967), Taylor & Vandyke (1985) and Dingerkus & Uhler (1977). These were measured and counted prior to clearing. Drawings were made with the assistance of a stereomicroscope. Osteological comparisons were made with repre�sentatives of marine Indo-Pacific atherinds and an atherionid (Table II). Specimens collected by H. Bleher�s party were deposited in the Australian Museum Sydney, those borrowed from the MNHN were returned to that institution. All of the specimens that have been examined are desig�nated as types.")

print([(ent.text, ent.label_ ,ent.start_char, ent.end_char, ent.ent_id_) for ent in paper.ents])


[('SL', 'Body_size', 1, 3, ''), ('marine', 'Distribution_descriptor', 6, 12, ''), ('length', 'Body_size', 27, 33, ''), ('eye diameter', 'Body_size', 38, 50, ''), ('marine', 'Distribution_descriptor', 492, 498, '')]


In [224]:
nlp_ner.pipe_names

['tok2vec', 'ner', 'entity_ruler']

In [223]:
#in order to put the entity_ruler into nlp_ner pipeline
#entity ruler's patterns(entities) evaluate only if they are not annotaded in training data, so the ”entity_ruler” will only add new entities that match to the patterns only if they don’t overlap with existing entities predicted by the statistical model
#QUEST PWS THA EPISTREFOUME TO LINK AN EINAI APO TRAINING ?PROS TO PARON TO EPISTREFOUME APO TO ENTITY RULER mono se auta pou den exoun oristei apo training loipon

if "entity_ruler" not in nlp_ner.pipe_names:
    ruler=nlp_ner.add_pipe("entity_ruler")

def dict_func(df,linkdf,label):
    patterns=[]
    j=0
    for i in df:
        dict={"label":label}
        dict["pattern"]=i #[{"LOWER" : str.lower(i)}]
        dict["id"]=linkdf[[1]][1][j]
        patterns.append(dict)
        j=j+1
    ruler.add_patterns(patterns)

dict_func(df_distr_descr,df,"Distribution Descriptor")
dict_func(df_life_stages,df1,"Life Stage")
dict_func(df_body_size,df2,"Body Size")
#dict_func(df_biol_descr,"Biological Descriptor")

In [225]:
doc=nlp_ner("Atherinids adult larval are small marine estuary and freshwaters Alpha-Polyhaline Body length (SL) standard length (sl) fishes not exceeding 150 mm SL (a soon to be described species of Craterocephalus may reach 300 mm SL), occurring predominantly in the Old World, with only Alepidomus evermanni (freshwaters of Cuba) and two marine species, Atherinomorus stipes and Hypoatherina harringtonensis predominantly in the shore waters of the Caribbean) known from the New World. In the Southeast Pacific, the family is only known only from Tumaco, Colombia, according to Dyer. All freshwater atherinid species are derived from marine ancestors and until recently have only been known from the rivers of Australia, New Guinea and some islands in the vicinity of the latter.")
paper=nlp_ner("(SL), marine Salinity head length and eye diameter. Counts were madeusingthetechniquerecommendedbyHubbs & Lagler (1958) so as to make comparisons with previously described atherinid species. Two speci�mens were cleared and stained usingthe method of Taylor (1967), Taylor & Vandyke (1985) and Dingerkus & Uhler (1977). These were measured and counted prior to clearing. Drawings were made with the assistance of a stereomicroscope. Osteological comparisons were made with repre�sentatives of marine Indo-Pacific atherinds and an atherionid (Table II). Specimens collected by H. Bleher�s party were deposited in the Australian Museum Sydney, those borrowed from the MNHN were returned to that institution. All of the specimens that have been examined are desig�nated as types.")

print([(ent.text, ent.label_ ,ent.start_char, ent.end_char, ent.ent_id_) for ent in paper.ents])



[('SL', 'Body_size', 1, 3, ''), ('marine', 'Distribution_descriptor', 6, 12, ''), ('Salinity', 'Distribution Descriptor', 13, 21, 'https://www.marinespecies.org/traits/wiki/Traits:Salinity'), ('length', 'Body_size', 27, 33, ''), ('eye diameter', 'Body_size', 38, 50, ''), ('marine', 'Distribution_descriptor', 492, 498, '')]


In [226]:
#psaxnw gia to pws tha valoume na epistrefei link me plhroforia
# nlp_ner.add_pipe("entity_linker", last=True)

# all_linked_entities = paper._.linkedEntities
# # iterates over sentences and prints linked entities
# for sent in doc.sents:
#     sent._.linkedEntities.pretty_print()